In [32]:
import requests
from bs4 import BeautifulSoup
import html2text
import mygene
import json
import pickle
import os
import pandas as pd
import numpy as np
mg = mygene.MyGeneInfo()

os.chdir("/storage10/shuchen/Genotype/GenePT/")

In [13]:
FP_SOSKIC = '/p/keles/Collab_2023/VolumeC/Soskic22_scRNA_count_matrix/'
FP_SOSKIC_CELL_META = FP_SOSKIC + "cell_meta_with_label_651650.csv"
FP_SOSKIC_GENE_META = FP_SOSKIC + "phase2_data_qced_cells_cellCycleScored_geneMetadata.csv"
FP_SOSKIC_GENE_ID_674 = FP_SOSKIC + "gene_ids_674_nr96.txt"
FP_SOSKIC_COMB_INFO = FP_SOSKIC + "combination_info_779.csv"

In [58]:
gene_meta = pd.read_csv(FP_SOSKIC_GENE_META)
gene_ids_674 = pd.read_csv(FP_SOSKIC_GENE_ID_674, header=None).iloc[:,0]

In [39]:
gene_names_674 = gene_meta["index"][gene_meta["gene_ids"].isin(gene_ids_674.iloc[:,0])]

In [59]:
gene_ids_674

0      ENSG00000187608
1      ENSG00000116251
2      ENSG00000215788
3      ENSG00000160049
4      ENSG00000028137
            ...       
669    ENSG00000198840
670    ENSG00000212907
671    ENSG00000198886
672    ENSG00000198786
673    ENSG00000198727
Name: 0, Length: 674, dtype: object

In [4]:
parts_to_remove = [
    "##  Summary\n",
    "NEW",
    'Try the newGene table',
    'Try the newTranscript table',
    '**',
    "\nGo to the top of the page Help\n"
]

def rough_text_from_gene_name(gene_number):
    
    # get url
    url = f"https://www.ncbi.nlm.nih.gov/gene/{gene_number}"
    # Send a GET request to the URL
    summary_text = ''
    soup = None
    try:
        response = requests.get(url, timeout=30)
    except requests.exceptions.Timeout:
        print('time out')
        return((summary_text,soup))
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the "summary" tab content by inspecting the page's structure
        summary_tab = soup.find('div', {'class': 'rprt-section gene-summary'})

        # Check if the "summary" tab content is found
        if summary_tab:
            # Convert the HTML to plain text using html2text
            html_to_text = html2text.HTML2Text()
            html_to_text.ignore_links = True  # Ignore hyperlinks

            # Extract the plain text from the "summary" tab
            summary_text = html_to_text.handle(str(summary_tab))
            # Remove the specified parts from the original text
            for part in parts_to_remove:
                summary_text = summary_text.replace(part, ' ')
                # Replace '\n' with a space
            summary_text = summary_text.replace('\n', ' ')

            # Reduce multiple spaces into one space
            summary_text = ' '.join(summary_text.split())
            # Print or save the extracted text
        else:
            print("Summary tab not found on the page.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    return((summary_text,soup))

In [71]:
# Using Gene CD24 as an example
# cd_24_name = mg.querymany('CD24', scopes='symbol', species='human')
# query_names_674 = mg.querymany(gene_names_674, scopes='symbol', species='human')
# mg.querymany(gene_names_674, scopes=["ensemblgene", "symbol", 'reporter', "ensembl.protein", "ensembl.transcript", "name", "alias"], species='human')
query_names_674 = mg.querymany(gene_ids_674, scopes=["ensembl.gene", "ensembl.transcript", "ensembl.protein"], species='human')

In [72]:
query_names_674[270]

{'query': 'ENSG00000273319',
 '_id': 'ENSG00000273319',
 '_score': 26.776138,
 'taxid': 9606}

In [65]:
query_names_674

[{'query': 'ENSG00000187608',
  '_id': '9636',
  '_score': 26.776133,
  'entrezgene': '9636',
  'name': 'ISG15 ubiquitin like modifier',
  'symbol': 'ISG15',
  'taxid': 9606},
 {'query': 'ENSG00000116251',
  '_id': '6146',
  '_score': 25.943548,
  'entrezgene': '6146',
  'name': 'ribosomal protein L22',
  'symbol': 'RPL22',
  'taxid': 9606},
 {'query': 'ENSG00000215788',
  '_id': '8718',
  '_score': 26.776152,
  'entrezgene': '8718',
  'name': 'TNF receptor superfamily member 25',
  'symbol': 'TNFRSF25',
  'taxid': 9606},
 {'query': 'ENSG00000160049',
  '_id': '1676',
  '_score': 26.776138,
  'entrezgene': '1676',
  'name': 'DNA fragmentation factor subunit alpha',
  'symbol': 'DFFA',
  'taxid': 9606},
 {'query': 'ENSG00000028137',
  '_id': '7133',
  '_score': 25.943542,
  'entrezgene': '7133',
  'name': 'TNF receptor superfamily member 1B',
  'symbol': 'TNFRSF1B',
  'taxid': 9606},
 {'query': 'ENSG00000070831',
  '_id': '998',
  '_score': 26.776152,
  'entrezgene': '998',
  'name': 'c

In [ ]:
gene_name_to_tax_id = {}
for i, result in enumerate(query_names_674):
    print(i)
    if i in [270]:
        continue
    if "_id" in result and "query" in result:
        gene_name_to_tax_id[result['symbol']] = result['_id']

In [85]:
with open("./data/soskic_674_ids.json", 'w') as file:
    json.dump(gene_name_to_tax_id, file)

In [86]:
with open('./data/soskic_674_ids.json', 'rb') as file:
    gene_name_to_tax_id = json.load(file)

In [87]:
gene_name_to_tax_id

{'ISG15': '9636',
 'RPL22': '6146',
 'TNFRSF25': '8718',
 'DFFA': '1676',
 'TNFRSF1B': '7133',
 'CDC42': '998',
 'ID3': '3399',
 'RPL11': '6135',
 'STMN1': '3925',
 'SH3BGRL3': '83442',
 'UBXN11': '91544',
 'CD52': '1043',
 'HMGN2': '3151',
 'LAPTM5': '7805',
 'ZCCHC17': '51538',
 'MARCKSL1': '65108',
 'ZBTB8OS': '339487',
 'ARMH1': '339541',
 'RPS8': '6202',
 'TUT4': '23318',
 'JUN': '3725',
 'ITGB3BP': '23421',
 'SRSF11': '9295',
 'ZRANB2': '9406',
 'TYW3': '127253',
 'GBP2': '2634',
 'RPAP2': '79871',
 'RPL5': '6125',
 'RNPC3': '55599',
 'C1orf162': '128346',
 'CD2': '914',
 'TXNIP': '10628',
 'MRPS21': '54460',
 'C1orf56': '54964',
 'CDC42SE1': '56882',
 'S100A10': '6281',
 'S100A11': '6282',
 'S100A6': '6277',
 'S100A4': '6275',
 'RPS27': '6232',
 'GPATCH4': '54865',
 'SH2D2A': '9047',
 'PYHIN1': '149628',
 'IFI16': '3428',
 'TAGLN2': '8407',
 'SELL': '6402',
 'RABGAP1L': '9910',
 'GLUL': '2752',
 'TPR': '7175',
 'PTPRC': '5788',
 'BTG2': '7832',
 'MDM4': '4194',
 'NUCKS1': '64710

In [16]:
# with open('./data/vocab.json', 'rb') as handle:
#     vocab_gene = json.load(handle)
# vocab_gene_list = list(vocab_gene.keys())

In [88]:
gene_name_to_summary_page = {}
for gene_name, page_id in sorted(gene_name_to_tax_id.items()):
    if gene_name not in gene_name_to_summary_page:
        print('gene_name',gene_name)
        parsed_text, unparsed_html = rough_text_from_gene_name(page_id)
        gene_name_to_summary_page[gene_name] = parsed_text

gene_name ABCF1
gene_name ABI3
gene_name ACTB
gene_name ACTG1
gene_name ACTN1
gene_name ADAL
gene_name ADI1
gene_name AIMP1
gene_name AKAP13
gene_name AKAP9
gene_name ALOX5AP
gene_name ANKRD11
gene_name ANKRD12
gene_name ANXA1
gene_name ANXA5
gene_name AP3M2
gene_name APOBEC3C
gene_name APOBEC3G
gene_name ARGLU1
gene_name ARID5B
gene_name ARL4C
gene_name ARMH1
gene_name ARRDC3
gene_name ATF7IP2
gene_name ATM
gene_name ATP5F1E
gene_name ATP5MG
gene_name ATP6
gene_name B2M
gene_name B4GALT1
gene_name BAX
gene_name BAZ1A
gene_name BDP1
gene_name BEX3
gene_name BEX5
gene_name BIRC2
gene_name BIRC3
gene_name BOLA3
gene_name BORCS7
gene_name BPTF
gene_name BRD3
gene_name BTG1
gene_name BTG2
gene_name BTN3A2
gene_name C12orf43
gene_name C12orf57
gene_name C12orf75
gene_name C16orf54
gene_name C16orf87
gene_name C1orf131
gene_name C1orf162
gene_name C1orf56
gene_name CALM1
gene_name CALR
gene_name CAMK4
gene_name CARD16
gene_name CARNMT1
gene_name CASP8AP2
gene_name CAST
gene_name CBR3
gene_na

In [89]:
len(gene_name_to_summary_page)

673

In [90]:
gene_name_to_summary_page

{'ABCF1': 'Official Symbol ABCF1provided by HGNC Official Full Name ATP binding cassette subfamily F member 1provided by HGNC Primary source HGNC:HGNC:70 See related Ensembl:ENSG00000204574 MIM:603429; AllianceGenome:HGNC:70 Gene type protein coding RefSeq status REVIEWED Organism Homo sapiens Lineage Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Hominidae; Homo Also known as ABC27; ABC50 Summary The protein encoded by this gene is a member of the superfamily of ATP-binding cassette (ABC) transporters. ABC proteins transport various molecules across extra- and intra-cellular membranes. ABC genes are divided into seven distinct subfamilies (ABC1, MDR/TAP, MRP, ALD, OABP, GCN20, White). This protein is a member of the GCN20 subfamily. Unlike other members of the superfamily, this protein lacks the transmembrane domains which are characteristic of most ABC transporters. This protein may be regulat

In [91]:
with open("./data/soskic_674_genes_summary_page.json", 'w') as file:
    json.dump(gene_name_to_summary_page, file)

### Experiment with Varied Gene Sets Tailored to Your Needs
#### For any specific gene name, mygene can be utilized to translate it into page IDs. We've illustrated an example using gene vocabularies in scGPT and Geneformer. Download links for these files are available in the repository's README.

In [12]:
# load genes used in GenePT
with open(f'./data/scGPT_vocab.json', 'rb') as handle:
    vocab_gene = json.load(handle)
vocab_gene_list = list(vocab_gene.keys())

# load genes used in Geneformer
with open(f"./data/geneformer_token_dictionary.pkl", 'rb') as handle:
    token_dictionary = pickle.load(handle)

# example query to convert gene IDs into page ids for NCBI 
vocab_gene_list_results = mg.querymany(sorted(vocab_gene_list), scopes='symbol', species='human')
token_dictionary_results = mg.querymany(sorted(token_dictionary.keys()), fields="symbol")

7092 input query terms found dup hits:	[('A2ML1-AS1', 2), ('A2ML1-AS2', 2), ('A2MP1', 2), ('AACSP1', 2), ('AADACL2-AS1', 3), ('AADACP1', 2)
21905 input query terms found no hit:	['5S_rRNA_ENSG00000276861', '5S_rRNA_ENSG00000277411', '5S_rRNA_ENSG00000277488', '5S_rRNA_ENSG00000
1 input query terms found dup hits:	[('ENSG00000268674', 3)]
3109 input query terms found no hit:	['<mask>', '<pad>', 'ENSG00000005955', 'ENSG00000006074', 'ENSG00000006075', 'ENSG00000006114', 'ENS


In [19]:
vocab_gene

{'RP5-973N23.5': 60693,
 'RP11-182N22.10': 60689,
 'CTB-53D8.3': 60687,
 'RP11-348N17.2': 60685,
 'RP11-205M20.8': 60682,
 'RP11-326C3.17': 60680,
 'RP11-439H13.3': 60675,
 'RP11-413H22.3': 60673,
 'GET1-SH3BGR': 60672,
 'CH17-476P10.1': 60670,
 'LLNLR-271E8.1': 60668,
 'CTD-2527I21.16': 60667,
 'CTC-359D24.5': 60664,
 'RP5-967N21.13': 60661,
 'CTC-264K15.5': 60659,
 'RP11-314O13.2': 60655,
 'RP11-127I20.9': 60652,
 'RP11-473M10.4': 60651,
 'OR7E11P_ENSG00000285537': 60644,
 'RP11-867O8.9': 60643,
 'TMX2-CTNND1': 60642,
 'RP11-732A19.10': 60641,
 'RP11-545E17.13': 60639,
 'RP11-1C8.8': 60638,
 'RP11-346I3.6': 60636,
 'GS1-273L24.6': 60635,
 'AL135749.6': 60634,
 'RP11-641J8.4': 60633,
 'RP11-478H11.3-001': 60631,
 'RP5-1111F22.1': 60629,
 'CTB-161K23.4': 60628,
 'RNF216P1_ENSG00000288620': 60625,
 'RP11-350J20.15': 60623,
 'XXbac-BPG254B15.11': 60622,
 'PDCD6-AHRR': 60618,
 'RP11-115L11.3': 60615,
 'RP11-553D4.3': 60614,
 'RP11-1020A11.3': 60613,
 'RP11-316O14.3': 60611,
 'AC012488.3':

In [20]:
vocab_gene_list_results

[{'query': '5S_rRNA_ENSG00000276861', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000277411', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000277488', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000278457', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000285609', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000285626', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000285674', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000285776', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000285912', 'notfound': True},
 {'query': '5S_rRNA_ENSG00000288601', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000273730', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000275757', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000275877', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000276871', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000277739', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000278294', 'notfound': True},
 {'query': '5_8S_rRNA_ENSG00000283274', 'notfound': True},
 {'query': '5_8S_

In [21]:
token_dictionary_results

[{'query': '<mask>', 'notfound': True},
 {'query': '<pad>', 'notfound': True},
 {'query': 'ENSG00000000003',
  '_id': '7105',
  '_score': 26.872438,
  'symbol': 'TSPAN6'},
 {'query': 'ENSG00000000005',
  '_id': '64102',
  '_score': 27.7306,
  'symbol': 'TNMD'},
 {'query': 'ENSG00000000419',
  '_id': '8813',
  '_score': 27.73056,
  'symbol': 'DPM1'},
 {'query': 'ENSG00000000457',
  '_id': '57147',
  '_score': 27.730627,
  'symbol': 'SCYL3'},
 {'query': 'ENSG00000000460',
  '_id': '55732',
  '_score': 27.730616,
  'symbol': 'FIRRM'},
 {'query': 'ENSG00000000938',
  '_id': '2268',
  '_score': 27.730633,
  'symbol': 'FGR'},
 {'query': 'ENSG00000000971',
  '_id': '3075',
  '_score': 27.730633,
  'symbol': 'CFH'},
 {'query': 'ENSG00000001036',
  '_id': '2519',
  '_score': 27.730707,
  'symbol': 'FUCA2'},
 {'query': 'ENSG00000001084',
  '_id': '2729',
  '_score': 27.73056,
  'symbol': 'GCLC'},
 {'query': 'ENSG00000001167',
  '_id': '4800',
  '_score': 27.730627,
  'symbol': 'NFYA'},
 {'query'